Ce code utilise la bibliothèque NLTK pour tokenizer le texte, filtre les mots inutiles à l'aide de la liste de mots vides français, et ensuite recherche les mots clés liés au développement informatique dans le texte tokenisé et filtré. Le résultat de l'exécution de ce code sera une liste de mots clés liés au développement informatique qui sont présents dans le texte donné.

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import nltk
import json
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# liste de mots clés liés au développement informatique
#skillpath = "/content/drive/MyDrive/Colab Notebooks/Projects/cleaned_skills.csv"
skillpath = "/content/drive/MyDrive/Colab Notebooks/Projects/cleaned_skills_small.csv"
skill_df = pd.read_csv(skillpath, sep=',', )

skill_df.head()

skill_list_set = set(skill_df['skills'])
print(len(skill_list_set))

1410


In [44]:
#jobspath = "/content/drive/MyDrive/Colab Notebooks/Projects/job_list.csv"
jobspath = "/content/drive/MyDrive/Colab Notebooks/Projects/job_list_paris.csv"
job_df = pd.read_csv(jobspath, sep=',', encoding='utf-16')

job_df.head()

,id,title,company,url,location,date,description
0,3427436026,Data Analyst,Foxintelligence,https://fr.linkedin.com/jobs/view/data-analyst...,"Paris, Île-de-France, France",2023-02-11,"\n\nChez Foxintelligence, nous sommes des amou..."
1,3479541484,Data Scientist H/F,AGYLIS,https://fr.linkedin.com/jobs/view/data-scienti...,"Boulogne-Billancourt, Île-de-France, France",2023-02-11,"\n AGYLIS, Société de services informat..."
2,3477087459,Creative Data Analyst - CDD,CHANEL,https://fr.linkedin.com/jobs/view/creative-dat...,"Neuilly-sur-Seine, Île-de-France, France",2023-02-10,\n\nCREATIVE DATA ANALYST (H/F) - CDD\nAu sein...
3,3479121451,Chef de projet AMOA Data management (F/H),Médecins du Monde,https://fr.linkedin.com/jobs/view/chef-de-proj...,"St.-Denis, Île-de-France, France",2023-02-11,\n\nTravailler avec Médecins du Monde \nVous s...
4,3473259005,Consultant(e) Data Strategy H/F,DataValue Consulting,https://fr.linkedin.com/jobs/view/consultant-e...,"Montrouge, Île-de-France, France",2023-02-11,\n Dans le cadre du développement de no...


In [45]:
# Mots à ignorer
stop_words_fr = set(stopwords.words('french'))
stop_words_en = set(stopwords.words('english'))
stop_words = stop_words_fr.union(stop_words_en)

def extract_technical_skills(description):
  # texte dans lequel on cherche les compétences techniques
  
  # text = "Je suis un développeur expérimenté en Java et Python, j'ai également de l'expérience en développement Agile et Scrum. J'ai également travaillé avec les outils DevOps tels que Linux et AWS."
  #text = "'\n\nKrut, cabinet de recrutement:recherche pour ELT/ETL son 'client' GCP, un acteur dans le marketing conversationnel, un data analyt, poste en full remote. \nAu sein de l’équipe IT composée d’une dizaine de profils de développeurs, vous interviendrez sur la mise en place, l’évolution et la maintenance des outils de reporting. En tant que premier profil BI de l’équipe, vous aurez un rôle de lead et serez force de proposition sur les sujets décisionnels, avec une grande liberté de mettre en place les outils que vous préconisez du côté ETL et Data visualisation/Dashboarding. D’une part, en collaboration avec les interlocuteurs métier et Customer Succes Managers, votre activité sera de définir, de mettre en place et maintenir les systèmes décisionnels.\nDans ce cadre vos missions seront :\n• Automatiser l’extraction, la modélisation, l’enrichissement des bases de données et la datavisualisation (ETL)\n• documenter, former et accompagner les utilisateurs dans l’analyse de leurs données\n• réaliser les reports et des tableaux de bord répondant aux attentes et aux besoins des utilisateurs\n• faire évoluer la conception et l’architecture du système décisionnel\n• Se construire une connaissance des métiers et des outils métiers\nD’autre part, vous collaborerez avec les dirigeants et managers d’eKonsilio afin de fournir des études décisionnelles et\nrecommandant pour faire évoluer le taux d’engagement des clients de la société :\n• Audit de l’existant\n• Enrichissement des référentiels de donnée\n• Challenge du besoin métier, recherche de nouvelles solutions et innovation data (données structurées venant de la\nplateforme web, des chatbots, mais aussi données non-structurées venant de sources variées)\n• Etude de faisabilité technique, ROI...\nLe data analyste reporte directement au Directeur Associé d’eKonsilio sur le suivi des réalisations, les problèmes et risques anticipés, la gestion de projet. Il escalade les difficultés rencontrées et participe à leur résolution avec le responsable\napplicatif.\nProfil recherché:\nDe formation Bac+5 Ecole d’Ingénieur informatique, vous avez déjà une expérience de 4 à 6 ans sur des projets data et décisionnel\nCompétences:\n• Expériences et connaissances des outils de création de reporting / data visualisation,\n• Expériences et connaissances technique des outils d’intégration type ETL/ELT\n• Bonne connaissance des langages SQL\n• Bonne connaissance des best practices de construction reporting, Datawarehouse, manipulation de données\n• Capacité d’analyse et de synthèse de sujets complexes\n• Capacités de communication et d’animation\n• Compréhension des contraintes technique IT : requête SQL, infrastructure, interface...\nPersonnalité / soft skills :\n• Autonomie et bonne organisation\n• Esprit d’équipe, vous aimez partager vos connaissances\n• Curiosité et esprit critique\n\n'"
  
  text = description
  text = text.replace('/', ' ')

  # Tokenisation
  tokens = nltk.word_tokenize(text)

  # On uniformise les mots en passant tout en minuscule
  lowered_tokens = [token.lower() for token in tokens]

  # On supprime les mots inutiles (stop words)
  filtered_tokens = [token for token in tokens if token not in stop_words]

  # On transforme la liste en set pour supprimer les doublons
  # Si une compétence est présente plusieurs fois dans une offre d'emploi, on la compte donc qu'une seule fois
  no_double_tokens = set(lowered_tokens)

  # Recherche des mots clés dans le texte
  technical_skills = [token for token in no_double_tokens if token in skill_list_set]

  return technical_skills

  #print(filtered_tokens)
  #print(technical_skills)
  #print(len(technical_skills), len(no_double_tokens), len(lowered_tokens))


#text = "'\n\nKrut, cabinet de recrutement:recherche pour ELT/ETL son 'client' GCP, un acteur dans le marketing conversationnel, un data analyt, poste en full remote. \nAu sein de l’équipe IT composée d’une dizaine de profils de développeurs, vous interviendrez sur la mise en place, l’évolution et la maintenance des outils de reporting. En tant que premier profil BI de l’équipe, vous aurez un rôle de lead et serez force de proposition sur les sujets décisionnels, avec une grande liberté de mettre en place les outils que vous préconisez du côté ETL et Data visualisation/Dashboarding. D’une part, en collaboration avec les interlocuteurs métier et Customer Succes Managers, votre activité sera de définir, de mettre en place et maintenir les systèmes décisionnels.\nDans ce cadre vos missions seront :\n• Automatiser l’extraction, la modélisation, l’enrichissement des bases de données et la datavisualisation (ETL)\n• documenter, former et accompagner les utilisateurs dans l’analyse de leurs données\n• réaliser les reports et des tableaux de bord répondant aux attentes et aux besoins des utilisateurs\n• faire évoluer la conception et l’architecture du système décisionnel\n• Se construire une connaissance des métiers et des outils métiers\nD’autre part, vous collaborerez avec les dirigeants et managers d’eKonsilio afin de fournir des études décisionnelles et\nrecommandant pour faire évoluer le taux d’engagement des clients de la société :\n• Audit de l’existant\n• Enrichissement des référentiels de donnée\n• Challenge du besoin métier, recherche de nouvelles solutions et innovation data (données structurées venant de la\nplateforme web, des chatbots, mais aussi données non-structurées venant de sources variées)\n• Etude de faisabilité technique, ROI...\nLe data analyste reporte directement au Directeur Associé d’eKonsilio sur le suivi des réalisations, les problèmes et risques anticipés, la gestion de projet. Il escalade les difficultés rencontrées et participe à leur résolution avec le responsable\napplicatif.\nProfil recherché:\nDe formation Bac+5 Ecole d’Ingénieur informatique, vous avez déjà une expérience de 4 à 6 ans sur des projets data et décisionnel\nCompétences:\n• Expériences et connaissances des outils de création de reporting / data visualisation,\n• Expériences et connaissances technique des outils d’intégration type ETL/ELT\n• Bonne connaissance des langages SQL\n• Bonne connaissance des best practices de construction reporting, Datawarehouse, manipulation de données\n• Capacité d’analyse et de synthèse de sujets complexes\n• Capacités de communication et d’animation\n• Compréhension des contraintes technique IT : requête SQL, infrastructure, interface...\nPersonnalité / soft skills :\n• Autonomie et bonne organisation\n• Esprit d’équipe, vous aimez partager vos connaissances\n• Curiosité et esprit critique\n\n'"
#extract_technical_skills(text)

# Limitation : ne fonctionne pas avec les compétences en deux mots comme "machine learning", "data mining", etc

In [46]:
def get_title(old_title):
  title = old_title.lower()
  if "data" in title:
    if "engineer" in title:
      return "Data Engineer"
    elif "analyst" in title:
      return "Data Analyst"
    elif "scientist" in title:
      return "Data Scientist"
    else:
      return "Data Autres"
  elif "bi" in title:
    return "Consultant BI"
  else:
    return "Autres"

def get_title_bis(old_title):
  old_title = old_title.lower().replace('é', 'e')
  new_title = []
  if "data" in old_title or "donnee" in old_title:
    new_title.append('Data')
    if "analyst" in old_title or "analyste" in old_title:
      new_title.append("Analyst") 
    elif "engineer" in old_title or "ingenieur" in old_title:
      new_title.append('Engineer')
    elif "scientist" in old_title or "scientifique" in old_title:
      new_title.append('Scientist')
    else:
      new_title.append('Others')
  else:
    new_title.append('Others')
  return " ".join(new_title)

get_title_bis('tott')

'Others'

In [47]:
job_df['skills'] = job_df['description'].map(lambda x : extract_technical_skills(x))
job_df['new_title'] = job_df['title'].map(get_title_bis)
job_df = job_df.drop(columns=['description'])

In [48]:
# On sauvegarde notre dataset dans un nouveau fichier pour l'utiliser dans l'analyse des description des jobs
savepath = "/content/drive/MyDrive/Colab Notebooks/Projects/job_list_with_skills.csv"
job_df.to_csv(savepath, sep =',', index=False)

savepath = "/content/drive/MyDrive/Colab Notebooks/Projects/job_list_with_skills.json"
job_df.to_json(savepath)